In [1]:
import pandas as pd

In [15]:
import urllib.request

url1 = 'https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv'
url2 = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz'

urllib.request.urlretrieve(url1, 'taxi+_zone_lookup.csv')
urllib.request.urlretrieve(url2, 'green_tripdata_2019-01.csv.gz')

('green_tripdata_2019-01.csv.gz', <http.client.HTTPMessage at 0x1ca21fe7290>)

In [19]:
greentrip_df = pd.read_csv('green_tripdata_2019-01.csv')
greentrip_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2,1,NaN


In [20]:
zone_df = pd.read_csv('taxi+_zone_lookup.csv')
zone_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [28]:
from sqlalchemy import create_engine

user = 'basit'
password = 'test123'
server = 'localhost'
database = 'ETLProjectDB'

# Replace 'ODBC Driver 17 for SQL Server' with the appropriate ODBC driver name if needed
engine = create_engine(f'mssql+pyodbc://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')

try:
    with engine.connect() as conn:
        print(f"Successfully connected to the database: {database}")
except Exception as ex:
    print(f"Sorry, could not connect to the database: {ex}")

Successfully connected to the database: ETLProjectDB


In [30]:
# Extracting table names present in the database
print(engine.table_names())

[]


C:\Users\92322\AppData\Local\Temp\ipykernel_9784\57297028.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


In [32]:
# create a new table and append data frame values to that table
zone_df.to_sql('zone_table', con=engine, if_exists='append',index=False)
greentrip_df.to_sql('greentrip_table', con=engine, if_exists='append',index=False)

-1

In [39]:
# Extracting table contents
result = engine.execute('select * from zone_table')
for row in result:
    print (row)
result.close()

(1, 'EWR', 'Newark Airport', 'EWR')
(2, 'Queens', 'Jamaica Bay', 'Boro Zone')
(3, 'Bronx', 'Allerton/Pelham Gardens', 'Boro Zone')
(4, 'Manhattan', 'Alphabet City', 'Yellow Zone')
(5, 'Staten Island', 'Arden Heights', 'Boro Zone')
(6, 'Staten Island', 'Arrochar/Fort Wadsworth', 'Boro Zone')
(7, 'Queens', 'Astoria', 'Boro Zone')
(8, 'Queens', 'Astoria Park', 'Boro Zone')
(9, 'Queens', 'Auburndale', 'Boro Zone')
(10, 'Queens', 'Baisley Park', 'Boro Zone')
(11, 'Brooklyn', 'Bath Beach', 'Boro Zone')
(12, 'Manhattan', 'Battery Park', 'Yellow Zone')
(13, 'Manhattan', 'Battery Park City', 'Yellow Zone')
(14, 'Brooklyn', 'Bay Ridge', 'Boro Zone')
(15, 'Queens', 'Bay Terrace/Fort Totten', 'Boro Zone')
(16, 'Queens', 'Bayside', 'Boro Zone')
(17, 'Brooklyn', 'Bedford', 'Boro Zone')
(18, 'Bronx', 'Bedford Park', 'Boro Zone')
(19, 'Queens', 'Bellerose', 'Boro Zone')
(20, 'Bronx', 'Belmont', 'Boro Zone')
(21, 'Brooklyn', 'Bensonhurst East', 'Boro Zone')
(22, 'Brooklyn', 'Bensonhurst West', 'Boro Zo

In [42]:
engine.dispose()